# ** RETO (opcional): Implementación algoritmo de descenso de gradiente en problema con múltiples minínimos locales **
**Facultad de ingeniería, departamento de Ingeniería Biomédica, Universidad de los Andes**\
**IBIO-2440 Programación científica**

**Nombres de los integrantes**


1.   David Tobón Molina
2.   David Santiago Rodríguez Quiroga

**Número del grupo**

Grupo 2

Considere el problema de optimización:



> $\min_x f(x)$ sujeto a $x\in\mathbb{R}^3$

donde $f:\mathbb{R^3}→\mathbb{R}$ y $x=[x_1,x_2,x_3]^T$. Note que este es un problema de minmización sin restricciones en $\mathbb{R}^3$. Se sabe que la función es derivable y posee varios minimizadores locales entre los cuales hayun minimizador global, todos con posiciones desconocidas. El objetivo del presente laboratorio bono es implementar y utilizar el algoritmo de descenso de gradiente para identificar la mayor cantidad de candidatos a minimizadores locales dentro de una región dada e identificar el posible minimizador global. 

*   Implemente su propia rutina del algoritmo de descenso de gradiente teniendo en cuenta que los minimizadores de la función están definida por $x_i ∈ [-5, 5]$ para cada $i = 1, …, 3$. Es decir, cada componente de $x$ està entre $-5$ y $5$. Defina un tamaño del salto $\alpha$ y un parámetro de criterio de parada $ϵ$ que usted considere conveniente para resolver el problema. Estime el gradiente de la función evaluado en un punto de forma numérica (como se hizo en una práctica anterior), y bajo ninguna circunstancia utilice variables simbólicas.
   
*    Corra el algoritmo de descenso de gradiente al menos para 500 condiciones iniciales diferentes, y guarde las soluciones resultantes de cada corrida (ya sea definidas de forma aleatoria o haciendo un recorrido definido en la región donde se sabe están los minimizadores locales). Basados en estos resultados, realice una tabla donde incluya los valores aproximados de los candidatos a minimizadores locales (recuerden que cada candidato es un vector 3-dimensional), el candidato a minimizador global, y el respectivo número de iteraciones que el algoritmo de descenso de gradiente requirió para llegar a la respuesta.

\\

Esta actividad funciona como un bono para la nota del primer parcial de los integrantes del grupo. La nota se asignará dependiendo de el correcto seguimiento de las instrucciones anteriores y la cantidad de minimizadores locales reales que su algoritmo encuentre. Cada grupo debe desarrollar su propia solución y no se permitirá copia entre grupos de todo el curso. Una falta ante esto se reportará inmediatamente ante el comité de ética de la facultad. 



In [6]:
import benchmark_functions as bf
import numpy as np
from tabulate import tabulate

# func es el objeto creado con la funcion a minimizar
func = bf.StyblinskiTang(n_dimensions=3)
print("Mínimo de la función", func.minimum())

Mínimo de la función (-117.4984971113142, [-2.903534, -2.903534, -2.903534])


In [7]:
# Dado un punto x0, func permite evaluar la función objetivo en x0 
x0 = np.array([5.0, 5.0, 5.0]) 
f=func(x0)
print(f)

375.0


In [8]:
def numeric_gradient(xi, f, e=0.000001):

    gradient_result = np.zeros(len(xi))

    for i in range(len(xi)):
        xi_k = np.copy(xi)
        xi_k[i] = xi[i]+e
        gradient_result[i] = (f(xi_k)-f(xi))/e

    return gradient_result


print("Gradiente incial:", numeric_gradient(x0, func))

Gradiente incial: [172.50006704 172.50006704 172.50006704]


In [9]:
def gradient_descent(f, numeric_gradient_func, xi, n_max=500):

    alpha = None
    e = None
    min_f = float(f(xi))
    x_min = np.copy(xi)

    # se prueban varios alphas y epsilons para encontrar el minimizador
    for i in range(1, 10):
        alpha = i/100
        for j in range(1, 10):
            
            # algoritmo de descenso del gradiente
            xk = np.copy(xi)
            e = j/10
            n = 0
            stop = False

            while not stop:

                num_grad = numeric_gradient_func(xk, f)
                xk1 = xk - (alpha*num_grad)

                if n+1 >= n_max:
                    stop = True
                elif np.linalg.norm(xk1 - xk) <= e:
                    stop = True
                else:
                    xk = np.copy(xk1)
                    n += 1
            
            # revisar si el punto encontrado por el algoritmo es el mejor para la condición inicial dada
            # actualizar los parámetros que obtuvieron el resultado
            if float(f(xk)) < min_f:
                min_f = float(f(xk))
                x_min = np.copy(xk)
                best_alpha = alpha
                best_epsilon = e
                best_n = n

    return best_n, best_alpha, best_epsilon, x_min, min_f

In [10]:
x0_values = [-5, 5]
x0 = np.zeros(3)
dict_for_best_approx = {}
table_headers = ['x0', 'n_iteraciones', 'alpha',
                 'epsilon', 'x_min: mínimo local', 'f(x_min)']
list_results = []
min_f = None

# probar el algoritmo para cada condición inicial y guardar información de cada resultado
for i in range(len(x0_values)):
    x0[0] = x0_values[i]
    for j in range(len(x0_values)):
        x0[1] = x0_values[j]
        for k in range(len(x0_values)):
            x0[2] = x0_values[k]
            n, a, e, x_min, min_f = gradient_descent(
                func, numeric_gradient, np.copy(x0))
            dict_for_best_approx[min_f] = (np.copy(x0), n, a, e, x_min)
            list_results.append([np.copy(x0), n, a, e, x_min, min_f])

print(tabulate(list_results, headers=table_headers, tablefmt="grid",
      maxcolwidths=50, floatfmt=(None, None, '.2f', '.2f', None, '.10f')))

# encontrar mejor aproximación
for key in dict_for_best_approx:
    if key < min_f:
        min_f = key

x_init, n, a, e, x_min = dict_for_best_approx[min_f]
print(
    f"\nCANDIDATO A MINIMIZADOR GLOBAL:\nx_init = {x_init}\nalpha = {a}\ne = {e}\nx = {x_min}\nmin_f = {min_f}\nn = {n}")

| x0            |   n_iteraciones |   alpha |   epsilon | x_min: mínimo local                |        f(x_min) |
|:--------------|----------------:|--------:|----------:|:-----------------------------------|----------------:|
| [-5. -5. -5.] |              11 |    0.03 |      0.10 | [-2.898378 -2.898378 -2.898378]    | -117.4971204334 |
| [-5. -5.  5.] |              11 |    0.03 |      0.10 | [-2.898378 -2.898378 -2.903535]    | -117.4975793257 |
| [-5.  5. -5.] |              11 |    0.03 |      0.10 | [-2.898378 -2.903535 -2.898378]    | -117.4975793257 |
| [-5.  5.  5.] |              11 |    0.03 |      0.10 | [-2.898378 -2.903535 -2.903535]    | -117.4980382185 |
| [ 5. -5. -5.] |              11 |    0.03 |      0.10 | [-2.903535 -2.898378 -2.898378]    | -117.4975793257 |
| [ 5. -5.  5.] |              11 |    0.03 |      0.10 | [-2.903535 -2.898378 -2.903535]    | -117.4980382185 |
| [ 5.  5. -5.] |              11 |    0.03 |      0.10 | [-2.903535 -2.903535 -2.898378]    | -